<a href="https://colab.research.google.com/github/Penetrati0n/fuzzy-sets-calculator/blob/feature%2Flab7/fuzzy_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Core

In [ ]:
import numpy as np
import pandas as pd

MATRIXS = {}

def parse_matrix(name: str, data: str):
    global MATRIXS
    MATRIXS[name] = np.matrix(data.replace(',', '.').strip().replace('\n', ';'))

def get_matrix(name: str):
    global MATRIXS
    return MATRIXS[name]

def new_matrix_shape(m1, m2) -> tuple:
    # s1 = m1.shape
    # s2 = m2.shape
    # return (min(s1[0], s2[0]), min(s1[1], s2[1]))
    return (m1 * m2).shape

def maximin(m1, m2):
    nm_shape = new_matrix_shape(m1, m2)
    nm = np.matrix(np.zeros(np.multiply(*nm_shape)).reshape(nm_shape))
    for i in range(nm_shape[0]):
        for j in range(nm_shape[1]):
            nm[i, j] = np.max(np.minimum(m1[i], m2[:, j].reshape(1, -1)))
    return nm

def minimax(m1, m2):
    nm_shape = new_matrix_shape(m1, m2)
    nm = np.matrix(np.zeros(np.multiply(*nm_shape)).reshape(nm_shape))
    for i in range(nm_shape[0]):
        for j in range(nm_shape[1]):
            nm[i, j] = np.min(np.maximum(m1[i], m2[:, j].reshape(1, -1)))
    return nm

def maximult(m1, m2):
    nm_shape = new_matrix_shape(m1, m2)
    nm = np.matrix(np.zeros(np.multiply(*nm_shape)).reshape(nm_shape))
    for i in range(nm_shape[0]):
        for j in range(nm_shape[1]):
            nm[i, j] = np.max(np.asarray(m1[i]) * np.asarray(m2[:, j].reshape(1, -1)))
    return nm

def parse_expressions(text: str) -> list:
    return [e.strip() for e in text.strip().split('\n') if e != '']

def parse_dimensions(group_text):
    lines = group_text.split('\n')
    number = int(lines[0].split('\t')[0])
    dimensions = {}
    for line in lines[1:]:
        args = line.split('\t')
        if '-' in args or ' -' in args:
            continue
        dimensions[args[0]] = tuple([int(i) for i in args[1:]])
    return (number, dimensions)
    

In [ ]:
# @title Polish notation

PRIORITY = {
    'maxm': 5,
    'minm': 5,
    'mult': 5,
}

ARGS = {
    'maxm': 2,
    'minm': 2,
    'mult': 2,
}

FUNCTIONS = {
    'maxm': maximin,
    'minm': minimax,
    'mult': maximult,
}

class Stack():
  __list: list

  def __init__(self):
    self.__list = []

  def push(self, el):
    self.__list.append(el)

  def pop(self):
    return self.__list.pop()

  def peek(self):
    return self.__list[-1] if len(self.__list) else None

  def empty(self) -> bool:
    return len(self.__list) == 0

  def view(self) -> list:
    return self.__list

def computing(text, sets):
  variables = []
  stack = Stack()

  for s in [el.strip() for el in text.replace('(', ' ( ').replace(')', ' ) ').split(' ') if el != '']:
    if s == '(':
      stack.push('(')
    elif s == ')':
      while stack.peek() != '(':
        variables.append(stack.pop())
      stack.pop()
    elif s in PRIORITY:
      if stack.empty() or stack.peek() == '(' or PRIORITY[stack.peek()] < PRIORITY[s]:
        stack.push(s)
      else:
        while not stack.empty() and stack.peek() != '(' and PRIORITY[stack.peek()] >= PRIORITY[s]:
          variables.append(stack.pop())
        stack.push(s)
        # variables.append(s)
    elif s in sets:
      variables.append(s)

  while not stack.empty():
    variables.append(stack.pop())

  stack = Stack()

  for el in variables:
    if el in sets:
      stack.push(sets[el])
    elif el in ARGS:
      if ARGS[el] == 1:
        el1 = stack.pop()
        stack.push(FUNCTIONS[el](el1))
      else:
        el2 = stack.pop()
        el1 = stack.pop()
        stack.push(FUNCTIONS[el](el1, el2))

  return stack.view()[0]

# Ввод данных

Ввод нечетких отношений (НО)

In [ ]:
# ------------------------------------------------------------------------------
# Матрица R1.
# ------------------------------------------------------------------------------
R1 = """
0,971108252	0,69342202	0,723531728	0,70513486	0,445972447	0,18490993	0,916186671	0,910459778	0,440809625
0,756438262	0,571855198	0,037271344	0,729664216	0,852754565	0,951775721	0,533756818	0,600996441	0,103953357
0,601652786	0,77034615	0,860034171	0,926187796	0,22604193	0,844910657	0,128528205	0,836032444	0,557351189
0,098693747	0,255953639	0,586626301	0,777263576	0,35479927	0,097412618	0,943247418	0,394043161	0,750751022
0,617399134	0,32245515	0,186833541	0,577499426	0,742604914	0,878111251	0,50781713	0,023955537	0,053056527
0,510930752	0,369165349	0,681184284	0,070153128	0,413019399	0,548886225	0,358530529	0,492608454	0,960214543
0,913062189	0,044920456	0,46738009	0,87862107	0,576606655	0,609940003	0,627558741	0,817492925	0,654506138
0,854183324	0,979174686	0,217055928	0,939858726	0,786416798	0,458711405	0,201024699	0,916357683	0,278541291
0,917119816	0,02415336	0,027741237	0,98192292	0,418370666	0,202086393	0,79185413	0,617597453	0,466867095

"""
# ------------------------------------------------------------------------------
# Матрица R2.
# ------------------------------------------------------------------------------
R2 = """
0,876166904	0,028540619	0,842934241	0,037740076	0,418733047	0,862190816	0,137198075	0,030185229	0,349122705
0,283370932	0,738778064	0,745848495	0,576893537	0,949940685	0,925111989	0,223605113	0,653633806	0,044780852
0,836000599	0,571072025	0,582748551	0,22247288	0,869128429	0,496661715	0,270344934	0,441410713	0,468455094
0,932473306	0,931139269	0,62654979	0,265293697	0,08676694	0,959009202	0,143072908	0,751391837	0,788952306
0,811571898	0,816443247	0,560441639	0,354865112	0,802660651	0,756231569	0,1095887	0,917653767	0,177579619
0,532692796	0,340226279	0,074212675	0,102629248	0,758317807	0,76473686	0,157305541	0,955853531	0,046525428
0,884118544	0,653694025	0,15941219	0,346272525	0,767395143	0,977322333	0,393735722	0,204724769	0,09904596
0,722572915	0,018077235	0,652011124	0,786839293	0,433820629	0,198548848	0,136159683	0,412033215	0,837190559
0,484339213	0,481135488	0,768411375	0,826516982	0,11288418	0,951474662	0,319082096	0,737652137	0,101184434

"""
# ------------------------------------------------------------------------------
# Матрица R3.
# ------------------------------------------------------------------------------
R3 = """
0,39424242	0,829844058	0,172821462	0,86181293	0,107988156	0,130913409	0,912483775	0,258954364	0,097697812
0,629540116	0,465299373	0,049048104	0,583119306	0,961129389	0,597935882	0,657185609	0,387015785	0,899576532
0,525568495	0,745853504	0,906405651	0,875663826	0,966469564	0,346284592	0,927546301	0,672966328	0,474718043
0,278114053	0,057601563	0,540809845	0,207937528	0,027190429	0,029743006	0,8761379	0,999493303	0,374880711
0,926224499	0,860759985	0,910089079	0,521097715	0,874355518	0,174866265	0,100851036	0,506619567	0,946733842
0,788142455	0,901219907	0,520006099	0,170475493	0,169331373	0,725868394	0,05189813	0,793513602	0,991798634
0,194778783	0,518578593	0,863374494	0,82653687	0,934266629	0,435021828	0,656195367	0,634369721	0,201352801
0,556887967	0,421602588	0,438108944	0,019305099	0,706981406	0,961636086	0,223055171	0,73096111	0,526255801
0,989487452	0,00447078	0,871497985	0,731539386	0,77481279	0,459849997	0,39955075	0,139403846	0,771746421

"""
# ------------------------------------------------------------------------------
# Матрица R4.
# ------------------------------------------------------------------------------
R4 = """
0,954711944	0,10763856	0,88827019	0,81494145	0,156039398	0,233676827	0,037944372	0,681359116	0,480914709
0,511506217	0,099687629	0,926493356	0,475067252	0,864902348	0,239985797	0,973513463	0,543963069	0,085016979
0,508624898	0,768837356	0,194238501	0,558370012	0,947420322	0,438370262	0,199612594	0,062410678	0,789042822
0,120300648	0,463239398	0,743765803	0,403524497	0,426986119	0,794862783	0,663275407	0,701483422	0,526731161
0,313082611	0,741946517	0,26556319	0,204432117	0,981360727	0,02562229	0,480721377	0,711548954	0,631273481
0,599762445	0,514420201	0,139568432	0,631512189	0,758025922	0,230849721	0,374833018	0,890925852	0,37056649
0,57750792	0,396341932	0,160218238	0,449899927	0,615328857	0,09362872	0,444634006	0,917643724	0,218119719
0,737148906	0,10798172	0,67270151	0,131630573	0,811791845	0,163418926	0,713254636	0,660430852	0,802016552
0,819453789	0,304192781	0,787476629	0,168616212	0,077648635	0,235871368	0,807096781	0,599850149	0,547990477
"""
# ------------------------------------------------------------------------------
# Всё, что ниже, не меняем.
# ------------------------------------------------------------------------------

parse_matrix('R1', R1)
parse_matrix('R2', R2)
parse_matrix('R3', R3)
parse_matrix('R4', R4)
print('НО загружены!')

НО загружены!


Ввод выражений

In [ ]:
# ------------------------------------------------------------------------------
# В expressions_text вставляем выражения из Excel.
# ------------------------------------------------------------------------------
expressions_text = """
( R2 maxm ( R3 minm R1 mult R1 ) ) 				
( R3 minm R3 ) 				
R1 maxm ( R2 minm R1 ) 				
( R2 maxm ( R3 minm R1 ) ) 				
R1 maxm ( R3 minm R2 ) 				
R3 minm R2 maxm ( R1 mult R2 ) 				
( R2 maxm R3 ) minm R1 				
( R2 maxm ( R1 minm R3 mult R3 ) ) 				
				
"""
# ------------------------------------------------------------------------------
# Всё, что ниже, не меняем.
# ------------------------------------------------------------------------------

expressions = parse_expressions(expressions_text)
print('Выражения загружены!')

Выражения загружены!


Ввод размерностей

In [ ]:
# ------------------------------------------------------------------------------
# В dimensions_text вставляем размерности НО из Excel.
# ------------------------------------------------------------------------------
dimensions_text = """
1	строки	столбцы
R1	5	5
R2	6	7
R3	7	5
R4	 -	 -
		
2	строки	столбцы
R1	 -	 -
R2	 -	 -
R3	7	7
R4	 -	 -
		
3	строки	столбцы
R1	5	8
R2	8	5
R3	 -	 -
R4	 -	 -
		
4	строки	столбцы
R1	7	6
R2	8	6
R3	6	7
R4	 -	 -
		
5	строки	столбцы
R1	6	7
R2	8	5
R3	7	8
R4	 -	 -
		
6	строки	столбцы
R1	8	6
R2	6	8
R3	5	6
R4	 -	 -
		
7	строки	столбцы
R1	8	5
R2	6	7
R3	7	8
R4	 -	 -
		
8	строки	столбцы
R1	6	8
R2	8	6
R3	8	8
R4	 -	 -

"""
# ------------------------------------------------------------------------------
# Всё, что ниже, не меняем.
# ------------------------------------------------------------------------------

dimensions = [parse_dimensions(g) for g in dimensions_text.strip().split('\n\t\t\n')]
print('Размерности загружены!')

Размерности загружены!


# Решение

In [ ]:
# @title Решение
for task_number, dimension in dimensions:
    matrixs_data = {}
    for d_name in dimension:
        row, col = dimension[d_name]
        matrixs_data[d_name] = MATRIXS[d_name][:row, :col]
    result = computing(expressions[task_number - 1], matrixs_data)
    result = np.c_[result, np.matrix(np.zeros(result.shape[0] * (10 - result.shape[1])).reshape(result.shape[0], -1))]
    result = np.vstack([result, np.matrix(np.zeros(10 * (10 - result.shape[0])).reshape(-1, 10))])
    result = np.array(result)
    print(f'{task_number}\tРазмерность результирующего НО')
    print('строки\t{}'.format(result.shape[0]))
    print('столбцы\t{}'.format(result.shape[1]))
    print('\t' + '\t'.join([str(i) for i in range(1, 11)]))
    idx = 1
    for i in range(result.shape[0]):
        print(f'{idx}\t' + '\t'.join(map(str, np.array(result[i]))).replace('.', ',').replace('\t0,0', '\t-1').replace('0,0\t', '-1\t'))
        idx += 1
    print()

1	Размерность результирующего НО
строки	6
столбцы	5
	1	2	3	4	5	6	7	8	9	10
1	0,734583438356738	0,5573698810676472	0,6222620098826774	0,6701262564923914	0,5913187930265212	-1	-1	-1	-1	-1
2	0,734583438356738	0,5573698810676472	0,6222620098826774	0,6701262564923914	0,5913187930265212	-1	-1	-1	-1	-1
3	0,5842699853233901	0,5573698810676472	0,582748551	0,6041386665763077	0,582748551	-1	-1	-1	-1	-1
4	0,62654979	0,5573698810676472	0,6222620098826774	0,62654979	0,5913187930265212	-1	-1	-1	-1	-1
5	0,5842699853233901	0,5573698810676472	0,560441639	0,6041386665763077	0,560441639	-1	-1	-1	-1	-1
6	0,532692796	0,45190531246409116	0,5045186644673315	0,6041386665763077	0,444368455277319	-1	-1	-1	-1	-1
7	-1	-1	-1	-1	-1	-1	-1	-1	-1	-1
8	-1	-1	-1	-1	-1	-1	-1	-1	-1	-1
9	-1	-1	-1	-1	-1	-1	-1	-1	-1	-1
10	-1	-1	-1	-1	-1	-1	-1	-1	-1	-1

2	Размерность результирующего НО
строки	7
столбцы	7
	1	2	3	4	5	6	7	8	9	10
1	0,39424242	0,745853504	0,39424242	0,170475493	0,169331373	0,174866265	0,107988156	-1	-1	-1
2	0,525568